<a href="https://colab.research.google.com/github/nehabachineni/AMR_GAT_EGT/blob/main/int1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


# Create directory
!mkdir -p /content/drive/MyDrive/colab_packages/IntegronFinder
%cd /content/drive/MyDrive/colab_packages/IntegronFinder

#  Install dependencies
!apt-get install -y hmmer infernal ncbi-blast+ prodigal
!pip install numpy biopython

#  Clone and install IntegronFinder
!git clone https://github.com/gem-pasteur/Integron_Finder.git
%cd Integron_Finder
!python3 setup.py install


/content/drive/MyDrive/colab_packages/IntegronFinder
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdivsufsort3 ncbi-data
Suggested packages:
  hmmer-doc
The following NEW packages will be installed:
  hmmer infernal libdivsufsort3 ncbi-blast+ ncbi-data prodigal
0 upgraded, 6 newly installed, 0 to remove and 30 not upgraded.
Need to get 23.2 MB of archives.
After this operation, 141 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 infernal amd64 1.1.4-1 [5,522 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libdivsufsort3 amd64 2.0.1-5 [42.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-data all 6.1.20170106+dfsg1-9 [3,519 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-blast+ amd64 2.12.0+ds-3build1 [12.3 MB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64

In [2]:
# Make sure directory exists (harmless if already exists)
!mkdir -p /content/drive/MyDrive/colab_packages/IntegronFinder

# Move into the directory
%cd /content/drive/MyDrive/colab_packages/IntegronFinder

# Install system dependencies again (needed after reconnect)
!apt-get install -y hmmer infernal ncbi-blast+ prodigal

# Install Python packages again
!pip install numpy biopython

# Move into the cloned Integron_Finder directory
%cd Integron_Finder

# Install IntegronFinder package
!python3 setup.py install


/content/drive/MyDrive/colab_packages/IntegronFinder
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdivsufsort3 ncbi-data
Suggested packages:
  hmmer-doc
The following NEW packages will be installed:
  hmmer infernal libdivsufsort3 ncbi-blast+ ncbi-data prodigal
0 upgraded, 6 newly installed, 0 to remove and 34 not upgraded.
Need to get 23.2 MB of archives.
After this operation, 141 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 infernal amd64 1.1.4-1 [5,522 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libdivsufsort3 amd64 2.0.1-5 [42.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-data all 6.1.20170106+dfsg1-9 [3,519 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-blast+ amd64 2.12.0+ds-3build1 [12.3 MB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64

In [3]:
from google.colab import drive
import os, glob, subprocess

drive.mount('/content/drive')

# Paths
input_dir = "/content/drive/MyDrive/poc/all_fastas_here"
output_dir = "/content/drive/MyDrive/IntegronFinder_results"
os.makedirs(output_dir, exist_ok=True)

fasta_files = sorted(glob.glob(f"{input_dir}/*.fna"))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


import os
import glob
import subprocess
import csv
from google.colab import drive

drive.mount('/content/drive')

input_dir = "/content/drive/MyDrive/poc/all_fastas_here"
output_dir = "/content/drive/MyDrive/IntegronFinder_results"
log_dir = os.path.join(output_dir, "logs")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

fasta_files = sorted(glob.glob(f"{input_dir}/*.fna"))
csv_output = os.path.join(output_dir, "integron_counts_first_half.csv")

# Clear CSV
with open(csv_output, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Strain", "Integrons_Found"])

for fasta in fasta_files[:362]:
    strain_id = os.path.basename(fasta).replace(".fna", "")
    outdir = os.path.join(output_dir, strain_id)
    log_file = os.path.join(log_dir, f"{strain_id}.log")
    os.makedirs(outdir, exist_ok=True)

    print(f" Running IntegronFinder on {strain_id}...")

    result = subprocess.run([
        "integron_finder",
        fasta,
        "--outdir", outdir,
        "--local-max",
        "--cpu", "2"
    ], capture_output=True, text=True)

    with open(log_file, "w") as logf:
        logf.write(result.stdout)
        logf.write(result.stderr)

    summary_file = os.path.join(outdir, f"{strain_id}.summary.tsv")
    if os.path.exists(summary_file):
        with open(summary_file) as sf:
            lines = sf.readlines()[1:]  # skip header
            count = len(lines)
    else:
        print(f"   {strain_id}: No summary file found.")
        count = 0

    with open(csv_output, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([strain_id, count])

    print(f"   {strain_id}: {count} integron(s) found.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Running IntegronFinder on 001.GCA_000401195.1_gkp2v01_genomic...
   001.GCA_000401195.1_gkp2v01_genomic: No summary file found.
   001.GCA_000401195.1_gkp2v01_genomic: 0 integron(s) found.
 Running IntegronFinder on 002.GCA_000406385.1_gkp47v01_genomic...
   002.GCA_000406385.1_gkp47v01_genomic: No summary file found.
   002.GCA_000406385.1_gkp47v01_genomic: 0 integron(s) found.
 Running IntegronFinder on 003.GCA_000406405.1_gkp1v01_genomic...
   003.GCA_000406405.1_gkp1v01_genomic: No summary file found.
   003.GCA_000406405.1_gkp1v01_genomic: 0 integron(s) found.
 Running IntegronFinder on 004.GCA_000406425.1_gkp24v01_genomic...
   004.GCA_000406425.1_gkp24v01_genomic: No summary file found.
   004.GCA_000406425.1_gkp24v01_genomic: 0 integron(s) found.
 Running IntegronFinder on 005.GCA_000406445.1_gkp25v01_genomic...
   005.GCA_000406445.1_gkp25v01_genomi

In [ ]:
# SYSTEM dependencies
!apt-get update
!apt-get install -y prodigal hmmer infernal

# Python dependencies
!pip install --upgrade pip
!pip install git+https://github.com/gem-pasteur/Integron_Finder.git


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,833 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packag

In [ ]:
!which integron_finder
!which prodigal
!which hmmsearch
!which cmsearch


/usr/local/bin/integron_finder
/usr/bin/prodigal
/usr/bin/hmmsearch
/usr/bin/cmsearch


In [ ]:
# Step 1: Install dependencies
!apt-get update
!apt-get install -y prodigal hmmer infernal

# Step 2: Install IntegronFinder
!pip install --upgrade pip
!pip install git+https://github.com/gem-pasteur/Integron_Finder.git

# Step 3: Check installations
!which integron_finder
!which prodigal
!which hmmsearch
!which cmsearch

# Step 4: Download test genome (Vibrio cholerae N16961)
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/006/745/GCF_000006745.1_ASM674v1/GCF_000006745.1_ASM674v1_genomic.fna.gz
!gzip -d GCF_000006745.1_ASM674v1_genomic.fna.gz

# Step 5: Run IntegronFinder on test genome
!integron_finder GCF_000006745.1_ASM674v1_genomic.fna --local-max --cpu 2 --outdir vibrio_test_out

# Step 6: Check results
!cat vibrio_test_out/*.summary.tsv || echo "No summary file found"


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [5]:
import os
import glob
import subprocess
import csv
from google.colab import drive

# Mount drive
drive.mount('/content/drive')

# Paths
input_dir = "/content/drive/MyDrive/poc/all_fastas_here"
output_dir = "/content/drive/MyDrive/IntegronFinder_results"
log_dir = os.path.join(output_dir, "logs")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

# Get list of fasta files
fasta_files = sorted(glob.glob(f"{input_dir}/*.fna"))
csv_output = os.path.join(output_dir, "integron_counts_first_half.csv")  # continue writing to same CSV

# DO NOT clear CSV this time ( already wrote first 362 entries)
# Just continue appending

# Process second half (from 363 to 725)
for fasta in fasta_files[362:]:
    strain_id = os.path.basename(fasta).replace(".fna", "")
    outdir = os.path.join(output_dir, strain_id)
    log_file = os.path.join(log_dir, f"{strain_id}.log")
    os.makedirs(outdir, exist_ok=True)

    print(f"Running IntegronFinder on {strain_id}...")

    # Run IntegronFinder
    result = subprocess.run([
        "integron_finder",
        fasta,
        "--outdir", outdir,
        "--local-max",
        "--cpu", "2"
    ], capture_output=True, text=True)

    # Save logs
    with open(log_file, "w") as logf:
        logf.write(result.stdout)
        logf.write(result.stderr)

    # Corrected summary file path
    result_folder = os.path.join(outdir, f"Results_Integron_Finder_{strain_id}")
    summary_file = os.path.join(result_folder, f"{strain_id}.summary")

    if os.path.exists(summary_file):
        with open(summary_file) as sf:
            lines = sf.readlines()[1:]  # skip header
            count = len(lines)
    else:
        print(f"   {strain_id}: No summary file found.")
        count = 0

    # Append results to CSV
    with open(csv_output, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([strain_id, count])

    print(f"   {strain_id}: {count} integron(s) found.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running IntegronFinder on 363.GCA_022299975.1_PDT000222827.2_genomic...
   363.GCA_022299975.1_PDT000222827.2_genomic: 88 integron(s) found.
Running IntegronFinder on 364.GCA_022299995.1_PDT000201871.2_genomic...
   364.GCA_022299995.1_PDT000201871.2_genomic: 114 integron(s) found.
Running IntegronFinder on 365.GCA_022300015.1_PDT000201870.2_genomic...
   365.GCA_022300015.1_PDT000201870.2_genomic: 73 integron(s) found.
Running IntegronFinder on 366.GCA_022300035.1_PDT000200515.3_genomic...
   366.GCA_022300035.1_PDT000200515.3_genomic: 75 integron(s) found.
Running IntegronFinder on 367.GCA_022300055.1_PDT000200502.3_genomic...
   367.GCA_022300055.1_PDT000200502.3_genomic: 60 integron(s) found.
Running IntegronFinder on 368.GCA_022300075.1_PDT000200505.3_genomic...
   368.GCA_022300075.1_PDT000200505.3_genomic: 161 integron(s) found.
Running IntegronFinder 

In [3]:
# Update package lists
!sudo apt-get update

# Install basic dependencies
!sudo apt-get install -y build-essential libncurses5-dev libncursesw5-dev zlib1g-dev libbz2-dev liblzma-dev python3 python3-pip perl

# Install BioPerl (required by Prokka)
!sudo cpan App::cpanminus
!sudo cpanm Bio::Perl

# Install Prokka via conda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -p /content/miniconda
!rm Miniconda3-latest-Linux-x86_64.sh
!/content/miniconda/bin/conda init
!source /content/miniconda/etc/profile.d/conda.sh
!/content/miniconda/bin/conda create -n prokka -c conda-forge -c bioconda prokka -y
!source /content/miniconda/bin/activate prokka

# Install IntegronFinder (since your pipeline uses it)
!/content/miniconda/bin/conda install -c bioconda integron_finder -y

Streaming output truncated to the last 5000 lines.










ncbi-vdb-3.2.1       | 10.6 MB   | :  50% 0.4974206524901646/1 [00:11<00:04,  9.20s/it] 
prokka-1.14.6        | 212.7 MB  | :  75% 0.7492249399430526/1 [00:11<00:04, 17.11s/it]










prokka-1.14.6        | 212.7 MB  | :  76% 0.755837044727802/1 [00:11<00:04, 16.53s/it] 
openjdk-22.0.1       | 173.2 MB  | :  95% 0.9521085808328658/1 [00:11<00:00, 13.86s/it]










ncbi-vdb-3.2.1       | 10.6 MB   | :  77% 0.7742589322986788/1 [00:11<00:00,  4.12s/it]
prokka-1.14.6        | 212.7 MB  | :  76% 0.7619348746959598/1 [00:11<00:04, 17.08s/it]










ncbi-vdb-3.2.1       | 10.6 MB   | :  93% 0.9252616303760501/1 [00:11<00:00,  2.89s/it]
prokka-1.14.6        | 212.7 MB  | :  77% 0.7687673829735342/1 [00:11<00:03, 16.32s/it]









hmmer-3.4            | 11.4 MB   | : 100% 1.0/1 [00:11<00:00,  2.49s/it]               
prokka-1.14.6        | 212.7 MB  | :  78% 0.7754529555892253/1 [00:12<00:03, 15.98s/it]











rpsbproc

In [4]:
# Check Prokka version
!prokka --version

/bin/bash: line 1: prokka: command not found


In [6]:
!source /content/miniconda/bin/activate prokka

In [7]:
!/content/miniconda/bin/conda install -c bioconda prokka -y

Channels:
 - bioconda
 - defaults
Platform: linux-64
Solving environment: - \ | / failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - nothing provides perl >=5.22.0,<5.23.0 needed by perl-bioperl-1.6.924-1

Could not solve for environment specs
The following packages are incompatible
├─ pin on python 3.12.* =* * is installable and it requires
│  └─ python =3.12 *, which can be installed;
└─ prokka =* * is not installable because there are no viable options
   ├─ prokka 1.11 would require
   │  └─ perl-bioperl =* * but there are no viable options
   │     ├─ perl-bioperl [1.7.2|1.7.8] would require
   │     │  └─ perl-bio-tools-run-alignment-tcoffee =* * but there are no viable options
   │     │     ├─ perl-bio-tools-run-alignment-tcoffee 1.7.4 would require
   │     │     │  └─ t_coffee =* * but there are no viable options
   │     │     │     ├─ t_coffee 11.0.8 would require
   │     │     │     │  └─ python [=2.7 *|>=2.7,<2.8.0a0 *], which conflicts w